Part of this notebook is taken from the tutorial-notebooks: 07b and 08 

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from lsst.rsp import get_tap_service, retrieve_query
import gc
import os

# Astropy
import astropy.units as u
from astropy.units import UnitsWarning
from astropy.timeseries import LombScargle

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom

In [6]:
%matplotlib inline

In [7]:
#initializing Tap and Butler
pd.set_option('display.max_rows', 20)
warnings.simplefilter("ignore", category=UnitsWarning)
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
params = {'axes.labelsize': 24,
          'font.size': 20,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [10, 8],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

In [9]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

## Plotting functions

In [10]:
# Plot 
def plotlc(days, magnitudes, band, out_name):
    fig = plt.figure(figsize=(8,8))
    plt.plot(days[band], magnitudes[band],
             'k.', ms=10)
    plt.minorticks_on()
    plt.xlabel('MJD (days)')
    plt.ylabel(band)
    plt.savefig(out_name)
    
    return fig
    #plt.show() 
    
def make_fig(cutout_image, ra, dec, out_name):
    """
    Create an image.
    should be followed with remove_figure

    Parameters
    ----------
    cutout_image : cutout_image from butler.get
    out_name : file name where you'd like to save it
    
    """
    fig = plt.figure(figsize=(4, 4))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(cutout_image.image)
    
    cutout_wcs = cutout_image.getWcs()
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    xy = geom.PointI(cutout_wcs.skyToPixel(radec))
    
    afw_display.dot('x', xy.getX(), xy.getY(), size=1, ctype='orange')
    plt.gca().axis('off')
    plt.savefig(out_name)
    
    return fig
    
    
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure

    gc.collect()    # call the garbage collector
    

## Creating directories to save lightcurves and coadds

In [24]:
!pwd

/home/jsv1206/Zooniverse/citizen-science-notebook


In [27]:
path = "/home/jsv1206/Zooniverse/citizen-science-notebook/variable_stars_output/coadd_iamges/"
isdir = os.path.isdir(path)
print(isdir)

False


In [13]:
plots = []
bands = ['g','r','i']
batch_dir = "./variable_stars_output"

if os.path.isdir(batch_dir) == False:
    os.mkdir(batch_dir)
    
if os.path.isdir(batch_dir+'/lc_plots') == False:
    os.mkdir(batch_dir+'/lc_plots')
else:
    os.system('rm -r '+batch_dir+'/lc_plots/*')
    
if os.path.isdir(batch_dir+'/text_files') == False:
    os.mkdir(batch_dir+'/text_files')
else:
    os.system('rm -r '+batch_dir+'/text_files/*')
    
if os.path.isdir(batch_dir+'/coadd_iamges') == False:
    os.mkdir(batch_dir+'/coadd_images')
# else:
#     os.system('rm -r '+batch_dir+'/coadd_images/*')

rm: cannot remove ‘./variable_stars_output/lc_plots/*’: No such file or directory
rm: cannot remove ‘./variable_stars_output/text_files/*’: No such file or directory


FileExistsError: [Errno 17] File exists: './variable_stars_output/coadd_images'

## Query to get variable stars

Selecting stars (truth_type=2)

variable (is_variable = 1)

is_pointsource = 1

In [9]:
def query_stars(ra_deg, dec_deg, radius_deg, limit):
    #radius within which you want to query
    
    query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec "\
            "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
            "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
            "WHERE ts.truth_type=2 "\
            "AND ts.is_variable = 1 "\
            "AND ts.is_pointsource = 1 "\
            "AND mt.match_objectId > 1 "\
            "AND CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', "+ str(ra_deg)+", "+str(dec_deg)+", "+str(radius_deg)+")) <= 1 "\
            "LIMIT "+str(limit)+" "
    results = service.search(query)
    variable_stars = results.to_table().to_pandas()
    return variable_stars

In [10]:
%%time
variable_stars = query_stars(62.1, -35.8, 0.10,10)

CPU times: user 44.7 ms, sys: 5.02 ms, total: 49.7 ms
Wall time: 1min 45s


In [11]:
variable_stars

,id_truth_type,match_objectId,ra,dec
0,40964951396_2,1405905936018577224,60.729425,-40.231732
1,40964949388_2,1405905936018578291,60.617284,-40.207596
2,40964948860_2,1405905936018578310,60.742047,-40.206829
3,40964950491_2,1405905936018578012,60.516994,-40.213474
4,41020236743_2,1405905936018577486,60.478843,-40.225947
5,41020236804_2,1405905936018577410,60.479612,-40.227918
6,40964950219_2,1405905936018577685,60.745899,-40.220974
7,40964950693_2,1405905936018577606,60.685522,-40.223228
8,40964946115_2,1405905936018579676,60.704483,-40.177513
9,40061862043_2,1405905936018579818,60.756003,-40.174776


## Query to get the flux at all the bands

In [12]:
def query_flux(objid):
    query = "SELECT src.band, src.ccdVisitId, src.coord_ra, src.coord_dec, "\
            "src.objectId, src.psfFlux, src.psfFluxErr, "\
            "scisql_nanojanskyToAbMag(psfFlux) as psfMag, "\
            "visinfo.band, "\
            "visinfo.expMidptMJD "\
            "FROM dp02_dc2_catalogs.ForcedSource as src "\
            "JOIN dp02_dc2_catalogs.CcdVisit as visinfo "\
            "ON visinfo.ccdVisitId = src.ccdVisitId "\
            "WHERE src.objectId = "+str(objid)+" "
    lc = service.search(query)
    flux_allbands = lc.to_table()
    # Create masks for the bands
    pick = {}
    for filter in plot_filter_labels:
        pick[filter] = (flux_allbands['band'] == filter)
    mjd_days = {}
    mags = {}
    for filter in plot_filter_labels:
        mjd_days[filter] = np.array(flux_allbands[pick[filter]]['expMidptMJD']) * u.day
        mags[filter] = np.array(flux_allbands[pick[filter]]['psfMag'])
        
    return mjd_days, mags

## Query coadd images from Butler

In [83]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collection)
skymap = butler.get('skyMap')
image_size = 100 #pixels

In [84]:
# From Melissa Graham's notebook and from Zooniverse testing
# https://github.com/rubin-dp0/cst-dev/blob/main/MLG_sandbox/DP02/citsci_sample_demo.ipynb

def get_cutout_image(ra_deg,dec_deg,band,datasetType='deepCoadd',cutoutSideLength=image_size):
    """
    get the tract and patch of a source. currently retrieves i band only. 

    Parameters
    ----------
    ra : ra of source in degrees
    dec : dec of source in degrees
    
    """
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
    
    spherePoint = geom.SpherePoint(ra_deg*geom.degrees, dec_deg*geom.degrees)
    tract = skymap.findTract(spherePoint)
    patch = tract.findPatch(spherePoint)
    xy = geom.PointI(tract.getWcs().skyToPixel(spherePoint))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    
    my_tract = tract.getId()
    my_patch = tract.getSequentialPatchIndex(patch)
    
    parameters = {'bbox': bbox}
    dataId = {'band': band, 'tract': my_tract, 'patch': my_patch}
    cutout_image = butler.get(datasetType, dataId=dataId, parameters=parameters)
    return cutout_image

In [88]:
for index, row in variable_stars.iterrows():
    for band in bands:
        deepCoadd= get_cutout_image(row['ra'],row['dec'],band,datasetType='deepCoadd')
        figout = make_figure(deepCoadd, row['ra'],row['dec'],batch_dir+"/coadd_images/"+str(row['id_truth_type'])+"_"+band+".png")
        remove_figure(figout)

## Plotting images and text files of the light curves

In [103]:
def plotlc(days, magnitudes, out_name):
    fig = plt.figure(figsize=(4, 3))
    for band in bands:
        plt.plot(days[band], magnitudes[band],
                 plot_filter_symbols[band], ms=10, label=band)
    plt.minorticks_on()
    plt.xlabel('MJD (days)')
    plt.ylabel('magnitude')
    #plt.legend('upper right')
    plt.savefig(out_name)
    return fig
    #plt.show() 

In [104]:
for index, row in variable_stars[0:2].iterrows():
    mjd_days, mags = query_flux(row['match_objectId'])
    figout = plotlc(mjd_days, mags, batch_dir+"/lc_plots/"+"lc_"+str(row['match_objectId'])+".png")
    remove_figure(figout) 

KeyboardInterrupt: 

In [91]:
%%time
for index, row in variable_stars.iterrows():
    df_all_bands = []
    mjd_days, mags = query_flux(row['match_objectId'])
    for band in bands:
        figout = plotlc(mjd_days, mags, band, batch_dir+"/lc_plots/"+"lc_"+str(row['match_objectId'])+'_'+band+".png")
        plots.append(figout)
        remove_figure(figout) 
        
        df = pd.DataFrame(data = {'band': [band]*len(mjd_days[band]), 'mjd_days': mjd_days[band], \
                          'mags': test_mags[band]}, index=None)
        df_all_bands.append(df)
    
    df_final = pd.concat(df_all_bands)
    outfile = batch_dir+"/text_files/"+"lc_"+str(row['match_objectId'])+".csv"
    df_final.to_csv(outfile, index=False, sep=',')

NameError: name 'test_mjd_days' is not defined